## Assignment 5 - Group 7

### First, we install all packages necesary and the chrome driver 

In [1]:
# Links 
# chromedriver= win 64: https://googlechromelabs.github.io/chrome-for-testing/
# chromedriver= win 32 :  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/120.0.6099.109/win32/chromedriver-win32.zip

In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver_manager==3.4.2

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Now, we import each library and install each package necesary to do the web scrapping
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [20]:
# Now, we install the extention in order to access to the page we want
driver = webdriver.Chrome()

# We also maximize the window
driver.maximize_window()

# We create an empty list to store DataFrames
all_table_data = []

# Here, we define the URL to be visited
url = 'https://infogob.jne.gob.pe/Eleccion'

# And open the specified URL in the Chrome browser
driver.get(url)

# Find and click on the dropdown element using XPath
dropdown = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
dropdown.click()

# Introduce a delay to wait for the dropdown options to load
time.sleep(1)

# Find and click on the option for presidential elections within the dropdown
elec_presidenciales = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
elec_presidenciales.click()

# Introduce a delay to wait for the next set of options to load
time.sleep(1)

# Find and click on the element corresponding to the election year
año_eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
año_eleccion.click()

# Introduce a delay to wait for the election year options to load
time.sleep(1)

# Find and click on the specific election year option using format
año_eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(2))
año_eleccion.click()

# Store the text content of the selected election year type
año_eleccion_tipo = año_eleccion.text

# Introduce a delay for stability before proceeding
time.sleep(1)


#####

# Get all elements of province
election_select = Select(driver.find_element(By.ID, "IdEleccion"))
num_election_options = len(election_select.options)

for elec_idx in range(1, num_election_options):
    # Get the election options again since HTML may be refreshing
    election_select = Select(driver.find_element(By.ID, "IdEleccion"))
    
    # Get the individual option
    election_option = election_select.options[elec_idx]

    # Get the option name of each election
    elec_name = election_option.get_attribute("text")
    
    print(elec_idx, elec_name)
    
    # Click the "Selecciona" button for each election
    selecciona = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    selecciona.click()
    time.sleep(2)

    # Navigate to the final page for each election
    pag_final = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    pag_final.click()
    time.sleep(2)
    
    # Obtain page's HTML
    page_source = driver.page_source
    
    # Use BeautifulSoup to analize HTML
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Find the table
    data_table = soup.find('table', {'class': 'table'})
    
    # Extract data from table and store it
    table_data = []
    for row in data_table.find_all('tr'):
        row_data = [cell.text for cell in row.find_all('td')]
        table_data.append(row_data)
        
    """
    
    table_path = driver.find_element(By.ID, 'gridEleccionResultadosElectorales')
    table_html = table_path.get_attribute ('innerHTML')
    table = pd.read_html(table_html)
    
    """
    
    # Append the table data to the list
    all_table_data.append(table_data)
    
    # Go back to the previous page
    driver.back()
    driver.back()

# After the loop, we create a DataFrame from all_table_data
flat_table_data = [item for sublist in all_table_data for item in sublist]

# Then we create the DataFrame
column_names = ['Organización Política', 'Símbolo', 'Plan de Gobierno', 'Total Votos', 'Porcentaje Votos Válidos', 'Lista de Candidatos']
df = pd.DataFrame(table_data, columns=column_names)
df


1 PRESIDENCIAL 2021 - 2DA VUELTA
2 PRESIDENCIAL 2021
3 PRESIDENCIAL 2016 - 2DA VUELTA
4 PRESIDENCIAL 2016
5 PRESIDENCIAL 2011 - 2DA VUELTA
6 PRESIDENCIAL 2011
7 PRESIDENCIAL 2006 - 2DA VUELTA
8 PRESIDENCIAL 2006
9 PRESIDENCIAL 2001 - 2DA VUELTA
10 PRESIDENCIAL 2001
11 PRESIDENCIAL 2000 - 2DA VUELTA
12 PRESIDENCIAL 2000
13 PRESIDENCIAL 1995
14 PRESIDENCIAL 1990 - 2DA VUELTA
15 PRESIDENCIAL 1990
16 PRESIDENCIAL 1985
17 PRESIDENCIAL 1980
18 PRESIDENCIAL 1963
19 PRESIDENCIAL 1962
20 PRESIDENCIAL 1956
21 PRESIDENCIAL 1950
22 PRESIDENCIAL 1945
23 PRESIDENCIAL 1939
24 PRESIDENCIAL 1936
25 PRESIDENCIAL 1931


,Organización Política,Símbolo,Plan de Gobierno,Total Votos,Porcentaje Votos Válidos,Lista de Candidatos
0,None,None,None,None,None,None
1,PARTIDO POLITICO NACIONAL PERU LIBRE,,,"8,836,380",50.126%,VER LISTA
2,FUERZA POPULAR,,,"8,792,117",49.874%,VER LISTA
3,VOTOS EN BLANCO,,,"121,489",,
4,VOTOS NULOS,,,"1,106,816",,


In [30]:
# Now, we install the extention in order to access to the page we want
driver = webdriver.Chrome()

# We also maximize the window
driver.maximize_window()

# We create an empty list to store DataFrames
all_table_data = []

# Here, we define the URL to be visited
url = 'https://infogob.jne.gob.pe/Eleccion'

# And open the specified URL in the Chrome browser
driver.get(url)

# Find and click on the dropdown element using XPath
dropdown = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
dropdown.click()

# Introduce a delay to wait for the dropdown options to load
time.sleep(1)

# Find and click on the option for presidential elections within the dropdown
elec_presidenciales = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
elec_presidenciales.click()

# Introduce a delay to wait for the next set of options to load
time.sleep(1)

# Find and click on the element corresponding to the election year
año_eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
año_eleccion.click()

# Introduce a delay to wait for the election year options to load
time.sleep(1)

# Find and click on the specific election year option using format
año_eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(2))
año_eleccion.click()

# Store the text content of the selected election year type
año_eleccion_tipo = año_eleccion.text

# Introduce a delay for stability before proceeding
time.sleep(1)


#####

# Get all elements of province
election_select = Select(driver.find_element(By.ID, "IdEleccion"))
num_election_options = len(election_select.options)

for elec_idx in range(1, num_election_options):
    # Get the election options again since HTML may be refreshing
    election_select = Select(driver.find_element(By.ID, "IdEleccion"))
    
    # Get the individual option
    election_option = election_select.options[elec_idx]

    # Get the option name of each election
    elec_name = election_option.get_attribute("text")
    
    print(elec_idx, elec_name)
    
    # Click the "Selecciona" button for each election
    selecciona = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    selecciona.click()
    time.sleep(2)

    # Navigate to the final page for each election
    pag_final = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    pag_final.click()
    time.sleep(2)
    
    # Obtain page's HTML
    page_source = driver.page_source
    
    # Use BeautifulSoup to analize HTML
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Find the table
    data_table = soup.find('table', {'class': 'table'})
    
    # Extract data from table and store it
    table_data = []
    for row in data_table.find_all('tr'):
        row_data = [cell.text for cell in row.find_all('td')]
        table_data.append(row_data)

    """
    
    We didn't use this code to extract the data since it failed to recognize the HTML within the table class.
    
    table_path = driver.find_element(By.ID, 'gridEleccionResultadosElectorales')
    table_html = table_path.get_attribute ('innerHTML')
    table = pd.read_html(table_html)
    
    """
    
    # Append the table data to the list of DataFrames
    column_names = ['Organización Política', 'Símbolo', 'Plan de Gobierno', 'Total Votos', 'Porcentaje Votos Válidos', 'Lista de Candidatos']
    df_iteration = pd.DataFrame(table_data, columns=column_names)
    all_table_data.append(df_iteration)
    
    # Go back to the previous page
    driver.back()
    driver.back()

# After the loop,we convert the list of DataFrames into a single DataFrame
df = pd.concat(all_table_data, ignore_index=True)
df


1 PRESIDENCIAL 2021 - 2DA VUELTA
2 PRESIDENCIAL 2021
                  Organización Política Símbolo Plan de Gobierno Total Votos  \
0                                  None    None             None        None   
1  PARTIDO POLITICO NACIONAL PERU LIBRE                            8,836,380   
2                        FUERZA POPULAR                            8,792,117   
3                       VOTOS EN BLANCO                              121,489   
4                           VOTOS NULOS                            1,106,816   
5                                  None    None             None        None   
6  PARTIDO POLITICO NACIONAL PERU LIBRE                            8,836,380   
7                        FUERZA POPULAR                            8,792,117   
8                       VOTOS EN BLANCO                              121,489   
9                           VOTOS NULOS                            1,106,816   

  Porcentaje Votos Válidos Lista de Candidatos  
0                

In [ ]:
# Now, we import each library and install each package necesary to do the web scrapping
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()

all_table_data = []


url = 'https://infogob.jne.gob.pe/Eleccion'

# And open the specified URL in the Chrome browser
driver.get(url)


for i in range(2,4):

    driver.get(url)
    dropdown = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
    dropdown.click()
    time.sleep(2)

    elec_presidenciales = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
    elec_presidenciales.click()
    time.sleep(1)

    año_eleccion=driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
    año_eleccion.click()
    time.sleep(1)

    año_eleccion = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(i))
    año_eleccion.click()
    año_eleccion_tipo = año_eleccion.text
    time.sleep(1)

    selecciona = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    selecciona.click()
    time.sleep(1)

    pag_final=driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    pag_final.click()
    time.sleep(2)
    
    # Obtain page's HTML
    page_source = driver.page_source
    
    # Use BeautifulSoup to analize HTML
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Find the table
    data_table = soup.find('table', {'class': 'table'})
    
    # Extract data from table and store it
    table_data = []
    for row in data_table.find_all('tr'):
        row_data = [cell.text for cell in row.find_all('td')]
        table_data.append(row_data)
        
    
    # Append the table data to the list
    all_table_data.append(table_data)
    
    # Go back to the previous page
    driver.back()
    driver.back()

# After the loop, we create a DataFrame from all_table_data
flat_table_data = [item for sublist in all_table_data for item in sublist]

# Then we create the DataFrame
column_names = ['Organización Política', 'Símbolo', 'Plan de Gobierno', 'Total Votos', 'Porcentaje Votos Válidos', 'Lista de Candidatos']
df = pd.DataFrame(table_data, columns=column_names)
df

